In [1]:
from func_datascope import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# idea for a layout -- FOR SHORT LINKS? :
# shortest paths lengths then try to find node groups closest to each other, 
# group then, make features from groups - to feature matrix with row are nodes, columns are group associations , then vis with umap - are links shorter?

In [3]:
path_plots = 'plots/'

In [4]:
# def sample_pts_spheresurface(n_samples, n_dimensions, radius=1):

#     gaussian = np.random.randn(n_samples, n_dimensions)
#     directions = gaussian / np.linalg.norm(gaussian, axis=1).reshape(n_samples, 1)
#     # For surface points, magnitudes are consistently equal to the sphere's radius
#     magnitudes = np.ones((n_samples, 1)) * radius
#     positions = magnitudes * directions
    
#     return positions

# N-DIM SPHERE

In [5]:
n_dimensions = [3,4,5,6] #,7,8,9,10,15,20,50,100,500,1000,5000,10000] #3,4,5,10,15,20,50,100,500,1000,5000,10000 

n_samples = 100  #6*n_dimensions # rule to thumb by felix : 6 for each dimension
layouttype = 'global' #globalmod' # 'spring' 'global' 'orig'
met = 'euclidean' # 'euclidean' # 'cosine'
radius = 1.0
NN = 10
sampled = 'volume' # 'surface' #'volume'

Graphs = {}
posG3D_graphs = {}

d_nodesizes_graphs = {}
d_nodecolors_graphs = {}

for j,n_dim in enumerate(n_dimensions):
    print(j)
    print("NDIM = ",n_dim)

    if sampled == 'volume':
        positions = sample_pts_spherevolume(n_samples,n_dim,radius)
        G = get_umap_network(positions,NN,met)
        d_pos_highdim = dict(zip(list(G.nodes()), positions))
        
    elif sampled == 'surface':
        positions = sample_pts_spheresurface(n_samples,n_dim)
        G = get_umap_network(positions,NN,met)
        d_pos_highdim = dict(zip(list(G.nodes()), positions))
        
    d_pos_highdim_norm = normalize_pos(d_pos_highdim)

    if n_dim == 3:
        posG3D_graphs[j] = d_pos_highdim_norm
        
        G.name = '0'+str(j)+'-Sphere-'+sampled+'_original3D'+'_'+str(len(G.nodes()))+'nodes'

        #d_nodecolors_original = dict(zip(list(G.nodes()), ['#D6D6D6']*len(G.nodes())))
        #d_nodesizes_original = dict(zip(list(G.nodes()), [3]*len(G.nodes())))
        #nx.set_node_attributes(G, d_nodecolors_original, name="nodecolor")
        #nx.set_edge_attributes(G, d_nodecolors_original[0], name="linkcolor")

        l_annotations_json = []
        for g in G.nodes():
            sublist = ["ID: "+str(g)]
            l_annotations_json.append(sublist) 
        d_annotations = dict(zip(G.nodes(), l_annotations_json))
        nx.set_node_attributes(G, d_annotations, name="annotation")

        
        #d_nodesizes_graphs[j] = d_nodesizes_original
        #d_nodecolors_graphs[j] = d_nodecolors_original
        print('Graph '+str(j)+' done')


    elif n_dim >= 4:

        ##############################################################

        # LAYOUT         

        ##############################################################
        
        if layouttype == 'spring':
            posG3D_spring = nx.spring_layout(G,dim=3,iterations = 200)
            posG3D = dict(zip(G.nodes(),[list(i) for i in posG3D_spring.values()]))

        elif layouttype == 'globalmod': # AIJ/SPL combined -> RWR 
            A = nx.adjacency_matrix(G)
            FM_adj = A.toarray()
            FM_spl = calculate_spl_matrix(G)

            # Normalize the feature matrices between 0 and 1 using Min-Max scaling
            min_max_scaler = MinMaxScaler()
            FM_spl_norm = min_max_scaler.fit_transform(FM_spl)
            matrix_combined =  combine_matrices(FM_adj, 0.5, FM_spl_norm, 0.5)

            # RWR - FEATURE MATRIX
            r = .8 
            alpha = 1.0  
            FM_rwrmod = random_walk_with_restart(matrix_combined, r, alpha, len(G.nodes()))
            columns = [i for i in range(FM_rwrmod.shape[1])]
            df_rwrmod = pd.DataFrame(data=FM_rwrmod, columns=columns)

            n_n = 10 # this should be constant
            spr = 1.0  
            md = 0.0  
            posG3D = layout_functional_umap(G,
                                        df_rwrmod,
                                        dim=3,
                                        n_neighbors=n_n, 
                                        spread=spr, 
                                        min_dist=md, 
                                        metric=met)

        elif layouttype == 'global':
            A = nx.adjacency_matrix(G)
            adj = A.toarray()
            DM_adj = pd.DataFrame(adj, columns = list(G.nodes()), index=list(G.nodes()))

            r = .8 # restart probability; originally 0.8
            alpha = 1.0 # alpha = 0.5 # --> better results for torus 
            DM_m = random_walk_with_restart(A, r, alpha, len(G.nodes()))
            DM_m_df = pd.DataFrame(DM_m)
            DM_m_transposed = DM_m_df.T

            DM_m_new = pd.DataFrame(distance.squareform(distance.pdist(DM_m_transposed, met)))
            DM_m_new.index = list(G.nodes())
            DM_m_new.columns = list(G.nodes())
            DM_m = DM_m_new

            n_n = 10 # this should be constant
            spr = 1.0  
            md = 0.0  
            posG3D = layout_functional_umap(G,
                                        DM_m_transposed, # DM_m
                                        dim=3,
                                        n_neighbors=n_n, 
                                        spread=spr, 
                                        min_dist=md, 
                                        metric=met)

        posG3D_graphs[j] = posG3D

    ##############################################################

    # VISUAL PROPERTIES      

    ##############################################################

    # NODE SIZES
    scale_factor_log = 2.25
    d_size_log = dict(draw_node_degree_log(G, scale_factor_log))
    d_nodesizes_graphs[j] = d_size_log

    # NODE COLORS    
    colvalue = 'dist2center'
    xdim_centroid = get_centroid_datacloud(d_pos_highdim_norm)
    
    d_xdim_centroid = {}
    d_xdim_centroid[0] = [i for i in xdim_centroid]
    
    l_distances_p2c_norm = calculate_distance_pts2center(n_dim,xdim_centroid,d_pos_highdim_norm)
    colors_cmap = matplotlib.cm.coolwarm_r #Blues_r #_r
    colors_cmap_sortedbyG = {}
    for i in l_distances_p2c_norm:
        colors_cmap_sortedbyG[i] = colors_cmap(i)
    colors_cmap_sortedbyG_hex = {}
    for dist,col in colors_cmap_sortedbyG.items():
        colors_cmap_sortedbyG_hex[dist] = matplotlib.colors.rgb2hex(col)  
    d_colors = dict(zip(list(G.nodes()),colors_cmap_sortedbyG_hex.values()))
    
    d_nodecolors_graphs[j] = d_colors
    
    G.name = '0'+str(j)+'-Sphere-'+sampled+'_'+str(n_dimensions[j])+'D'+'_'+str(len(G.nodes()))+'nodes'
    nx.set_node_attributes(G, d_nodecolors_graphs[j], name="nodecolor")
    #nx.set_edge_attributes(G, d_nodecolors_graphs[j][0], name="linkcolor")
    nx.set_edge_attributes(G, '#7C7C7C', name="linkcolor")


    # NODE ANNOTATION
    d_degree = dict(G.degree())
    l_annotations_json = []
    for g in G.nodes():
        sublist = ["ID: "+str(g)+"/" +"degree: "+str(d_degree[g])]
        l_annotations_json.append(sublist)
            
    d_annotations = dict(zip(G.nodes(), l_annotations_json))
    nx.set_node_attributes(G, d_annotations, name="annotation")

    Graphs[j] = G
    

    print('Graph '+str(j)+' done')
    print("Graph nodes:", len(G.nodes()))
    print("Graph edges:", len(G.edges()))

0
NDIM =  3
Graph 0 done
Graph 0 done
Graph nodes: 100
Graph edges: 548
1
NDIM =  4
Graph 1 done
Graph nodes: 100
Graph edges: 550
2
NDIM =  5
Graph 2 done
Graph nodes: 100
Graph edges: 550
3
NDIM =  6
Graph 3 done
Graph nodes: 100
Graph edges: 568


# adapt layouts with ICP 

In [6]:
import open3d as o3d

# adapt/transform layouts with ICP
posG3D_graphs_new = []
posG3D_t = posG3D_graphs[0]
posG3D_graphs_new.append(posG3D_t)

for i in range(1, len(posG3D_graphs)):
    posG3D_t = perform_icp(posG3D_t, posG3D_graphs[i])
    posG3D_graphs_new.append(posG3D_t)

for x,G in enumerate(Graphs): 
    nx.set_node_attributes(Graphs[x], posG3D_graphs_new[x], name="pos")

d_nodecolors = d_nodecolors_graphs

In [7]:
dict1 = posG3D_graphs_new[0] 
dict2 = posG3D_graphs_new[1]

G_byID = nx.Graph()
for node_id in dict1.keys():
    G_byID.add_node(node_id)
    G_byID.add_edge(node_id, node_id)
positions = {node_id: (dict1[node_id],dict2[node_id]) for node_id in dict1.keys()}

edge_x = []
edge_y = []
edge_z = []

for edge in G_byID.edges():
    x0, y0, z0 = positions[edge[0]][0]
    x1, y1, z1 = positions[edge[0]][1]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])

edge_trace_byID = pgo.Scatter3d(
    x=edge_x,
    y=edge_y,
    z=edge_z,
    mode='lines',
    line=dict(color='#FFFFFF', width=1),
)

In [8]:
# opacity_nodes = 0.9
# node_edge_col = '#696969' 
# node_linewidth = 0.25

# edge_opacity = 0.1
# edge_width = 2
# edge_color = '#696969'  ##7D7D7D' # '#d3d3d3'

# node_traces = []
# for i in range(len(posG3D_graphs_new)):
    
#     node_traces.append(get_trace_nodes_3D_legend(posG3D_graphs_new[i], 
#                                                 dict(zip(list(Graphs[i].nodes()),posG3D_graphs_new[i].keys())),
#                                                 d_nodecolors[i], #d_nodecolors_graphs[i], #dict(zip(list(Graphs[i].nodes()),['#0080FF']*len(Graphs[i].nodes()))), # blue
#                                                 d_nodesizes_graphs[i] 
#                                                 )
#                         )

# edge_traces = []
# for i in range(len(posG3D_graphs_new)):
    
#     edge_traces.append(get_trace_edges_3D(Graphs[i], 
#                                                 posG3D_graphs_new[i],
#                                                 d_nodecolors[i][0], #d_nodecolors_graphs[i], #dict(zip(list(Graphs[i].nodes()),['#0080FF']*len(Graphs[i].nodes()))), # blue
#                                                 opac = 0.5,
#                                                 linewidth = 1 #0.5
#                                                 )
#                         )




# fig = pgo.Figure()
# for x,i in enumerate(node_traces):
#     for j in i:
#         fig.add_trace(j)
#     #fig.add_trace(i)

# for i in edge_traces:
#     fig.add_trace(i)

# #fig.add_trace(edge_trace_byID)

# fig.update_layout(template='plotly_dark', 
#                  showlegend=True, width=1200, height=1200,
#                         scene=dict(
#                             xaxis_title='',
#                             yaxis_title='',
#                             zaxis_title='',
#                             xaxis=dict(nticks=0,tickfont=dict(
#                                 color='black')),
#                             yaxis=dict(nticks=0,tickfont=dict(
#                                 color='black')),
#                             zaxis=dict(nticks=0,tickfont=dict(
#                                 color='black')),
#                         dragmode="turntable",
#                     ))

# fig.update_xaxes(visible=False)
# fig.update_yaxes(visible=False)

# fig.write_html(path_plots+'NdimSPHERE.html')
# fig.show()

In [9]:
# node colors by dimension - for plotting
colormap = plt.cm.get_cmap('coolwarm')
color_indices = np.linspace(0, 1, len(posG3D_graphs_new))
colors = [colormap(index) for index in color_indices]
hex_colors = [mcolors.to_hex(c) for c in colors]
color_indices = np.arange(len(hex_colors))

#plt.figure(figsize=(10, 2)) 
#plt.bar(color_indices, height=1, color=hex_colors, edgecolor='black')

d_nodecolors_perlayout = {}
for x in range(len(posG3D_graphs_new)):
    d_nodecolors_perlayout[x] = dict(zip(Graphs[x].nodes(),[hex_colors[x]]*len(Graphs[x].nodes())))
    

In [10]:
opacity_nodes = 0.9
node_edge_col = '#696969' 
node_linewidth = 0.25

edge_opacity = 0.1
edge_width = 2
edge_color = '#696969'  ##7D7D7D' # '#d3d3d3'

node_traces = []
for i in range(len(posG3D_graphs_new)):
    
    node_traces.append(get_trace_nodes_3D_legend(posG3D_graphs_new[i], 
                                                dict(zip(list(Graphs[i].nodes()),posG3D_graphs_new[i].keys())),
                                                d_nodecolors_perlayout[i], #d_nodecolors_graphs[i], #dict(zip(list(Graphs[i].nodes()),['#0080FF']*len(Graphs[i].nodes()))), # blue
                                                d_nodesizes_graphs[i] 
                                                )
                        )

edge_traces = []
for i in range(len(posG3D_graphs_new)):
    
    edge_traces.append(get_trace_edges_3D(Graphs[i], 
                                                posG3D_graphs_new[i],
                                                d_nodecolors_perlayout[i][0], #d_nodecolors_graphs[i], #dict(zip(list(Graphs[i].nodes()),['#0080FF']*len(Graphs[i].nodes()))), # blue
                                                opac = 0.5,
                                                linewidth = 2 #0.5
                                                )
                        )




fig = pgo.Figure()
for x,i in enumerate(node_traces):
    fig.add_trace(*i)

for i in edge_traces:
    fig.add_trace(i)

#fig.add_trace(edge_trace_byID)

fig.update_layout(template='plotly_dark', 
                 showlegend=True, width=1200, height=1200,
                        scene=dict(
                            xaxis_title='',
                            yaxis_title='',
                            zaxis_title='',
                            xaxis=dict(nticks=0,tickfont=dict(
                                color='black')),
                            yaxis=dict(nticks=0,tickfont=dict(
                                color='black')),
                            zaxis=dict(nticks=0,tickfont=dict(
                                color='black')),
                        dragmode="turntable",
                    ))

fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)

fig.write_html(path_plots+'NdimSPHERE.html')
fig.show()

# TO VR 
new json format 

In [11]:
# contains the merge nx graph function to format them into one json file in the structure required for the DataDiVR
import nx2json as nx2j

In [12]:
# this is where my Graphs are stored (essentially a list of nx.Graph objects)
Graphs_for_VR = list(Graphs.values())
Graphs_for_VR

In [13]:
# set "projectname" (optional)
projectname = 'NDIM_Sphere'
for G in Graphs_for_VR:
    G.graph['projectname'] = projectname

In [14]:
G_merged = nx2j.merge_graphs(Graphs_for_VR)

Current working directory:  /Users/chris/Desktop/Github/DataScope
Merged JSON file saved as:  /Users/chris/Desktop/Github/DataScope/NDIM_Sphere.json


# EXAMINATION tests

In [12]:
# same positions - different links 
same = 0 # i.e. 3D 

opacity_nodes = 0.9
node_edge_col = '#696969' 
node_linewidth = 0.25

edge_opacity = 0.1
edge_width = 2
edge_color = '#696969'  ##7D7D7D' # '#d3d3d3'

node_traces = []
for i in range(len(posG3D_graphs_new)):
    
    node_traces.append(get_trace_nodes_3D_legend(posG3D_graphs_new[same], 
                                                dict(zip(list(Graphs[i].nodes()),posG3D_graphs_new[i].keys())),
                                                d_nodecolors_perlayout[i], #d_nodecolors_graphs[i], #dict(zip(list(Graphs[i].nodes()),['#0080FF']*len(Graphs[i].nodes()))), # blue
                                                d_nodesizes_graphs[i] 
                                                )
                        )

edge_traces = []
for i in range(len(posG3D_graphs_new)):
    
    edge_traces.append(get_trace_edges_3D(Graphs[i], 
                                                posG3D_graphs_new[same],
                                                d_nodecolors_perlayout[i][0], #d_nodecolors_graphs[i], #dict(zip(list(Graphs[i].nodes()),['#0080FF']*len(Graphs[i].nodes()))), # blue
                                                opac = 0.5,
                                                linewidth = 2 #0.5
                                                )
                        )




fig = pgo.Figure()
for x,i in enumerate(node_traces):
    fig.add_trace(*i)

for i in edge_traces:
    fig.add_trace(i)

#fig.add_trace(edge_trace_byID)

fig.update_layout(template='plotly_dark', 
                 showlegend=True, width=1200, height=1200,
                        scene=dict(
                            xaxis_title='',
                            yaxis_title='',
                            zaxis_title='',
                            xaxis=dict(nticks=0,tickfont=dict(
                                color='black')),
                            yaxis=dict(nticks=0,tickfont=dict(
                                color='black')),
                            zaxis=dict(nticks=0,tickfont=dict(
                                color='black')),
                        dragmode="turntable",
                    ))

fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)

fig.write_html(path_plots+'NdimSPHERE.html')
fig.show()


In [13]:
check_float32_in_graphs(Graphs)
convert_float32_attributes(Graphs)

Graph 0, Edge (0, 3), Attribute weight contains float32
Graph 0, Edge (0, 4), Attribute weight contains float32
Graph 0, Edge (0, 15), Attribute weight contains float32
Graph 0, Edge (0, 20), Attribute weight contains float32
Graph 0, Edge (0, 23), Attribute weight contains float32
Graph 0, Edge (0, 48), Attribute weight contains float32
Graph 0, Edge (0, 49), Attribute weight contains float32
Graph 0, Edge (0, 56), Attribute weight contains float32
Graph 0, Edge (0, 57), Attribute weight contains float32
Graph 0, Edge (0, 73), Attribute weight contains float32
Graph 0, Edge (0, 79), Attribute weight contains float32
Graph 0, Edge (0, 91), Attribute weight contains float32
Graph 0, Edge (0, 94), Attribute weight contains float32
Graph 0, Edge (0, 97), Attribute weight contains float32
Graph 0, Edge (1, 37), Attribute weight contains float32
Graph 0, Edge (1, 41), Attribute weight contains float32
Graph 0, Edge (1, 45), Attribute weight contains float32
Graph 0, Edge (1, 64), Attribute 

# EXPORT - VR 

In [14]:
for ix, graph in enumerate(Graphs):
    G_json = json.dumps(graph_to_serializable(Graphs[ix]), indent=4)
    filename = f"VR/"+Graphs[ix].name+".json"
    with open(filename, "w") as outfile:
        outfile.write(G_json)

# Difference of node degree distribution in between dimensions 

# Distribution of Distances 

In [ ]:
l_n_dimensions = [1,2,3,5,10,50,100,500,1000,5000,10000]
n_samples = 5000
radius = 1.0
NN = 10 

all_distance_lists = []
for i in l_n_dimensions: 
    
    print(i)
    n_dimensions = i
    positions = sample_pts_spherevolume(n_samples,n_dimensions,radius)
    G = get_umap_network(positions,NN,met)
    d_pos_highdim = dict(zip(list(G.nodes()), positions))
    d_pos_highdim_norm = normalize_pos(d_pos_highdim)

    xdim_centroid = get_centroid_datacloud(d_pos_highdim_norm)
    d_xdim_centroid = {}
    d_xdim_centroid[0] = [i for i in xdim_centroid]

    l_distances_p2c_norm = calculate_distance_pts2center(n_dimensions,xdim_centroid,d_pos_highdim_norm)      
    all_distance_lists.append(l_distances_p2c_norm)
    
    print('finished dim.')

In [ ]:
l_colors_cmap = matplotlib.cm.get_cmap('Blues', len(all_distance_lists))
l_colors = []
for i in range(l_colors_cmap.N):
    rgb = l_colors_cmap(i)

    l_colors.append(matplotlib.colors.to_hex(rgb))

plot_colorlegend(l_colors_cmap)

num_bins = 50

fig, ax = plt.subplots(figsize=(8,8),dpi=300)

for ix,l_dist in enumerate(all_distance_lists):
    n, bins, patches = ax.hist(l_dist, bins=num_bins, density=True,color=l_colors[ix],alpha=0.2)
    mu, sigma = scipy.stats.norm.fit(l_dist)
    best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
    plt.plot(bins, best_fit_line, c=l_colors[ix])
    
ax.set_xlabel('Distances (points to center)',fontsize=16)
ax.set_xlim(0,1)
ax.set_ylabel('Probability density',fontsize=16)

plt.suptitle(r'Distance Distribution',fontsize=18)
plt.title(r'from Points to Center | ' + sampled +' sampled | # samples: '+str(n_samples), fontsize=12)

ax.legend(labels=[str(i)+' Dim.' for i in l_n_dimensions])
fig.tight_layout()

plt.savefig(path_plots+'Dist_distributions_'+sampled+'_nbims'+str(num_bins)+'_nsamp'+str(n_samples)+'.png')
plt.show()

# code snippets

### link traces from same nodeIDs in different layouts to follow trajectory: 

In [ ]:
dict1 = posG3D_graphs_new[0] 
dict2 = posG3D_graphs_new[1]

G_byID = nx.Graph()
for node_id in dict1.keys():
    G_byID.add_node(node_id)
    G_byID.add_edge(node_id, node_id)
positions = {node_id: (dict1[node_id],dict2[node_id]) for node_id in dict1.keys()}

edge_x = []
edge_y = []
edge_z = []

for edge in G_byID.edges():
    x0, y0, z0 = positions[edge[0]][0]
    x1, y1, z1 = positions[edge[0]][1]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])

# edge_trace_byID = pgo.Scatter3d(
#     x=edge_x,
#     y=edge_y,
#     z=edge_z,
#     mode='lines',
#     line=dict(color='#FFFFFF', width=3),
# )

# start_x, start_y, start_z = [i[0][0] for i in positions.values()], [i[0][1] for i in positions.values()], [i[0][2] for i in positions.values()]
# end_x, end_y, end_z = [i[1][0] for i in positions.values()], [i[1][1] for i in positions.values()], [i[1][2] for i in positions.values()]

# start_node_trace = pgo.Scatter3d(
#     x=start_x,
#     y=start_y,
#     z=start_z,
#     mode='markers',
#     marker=dict(size=6, color='blue'),
# )

# end_node_trace = go.Scatter3d(
#     x=end_x,
#     y=end_y,
#     z=end_z,
#     mode='markers',
#     marker=dict(size=6, color='red'),
# )

### sphere sampling - layout - vis without ICP 

In [ ]:
n_dimensions = 3 #3,4,5,10,15,20,50,100,500,1000,5000,10000 
n_samples = 1000 #6*n_dimensions # rule to thumb by felix : 6 for each dimension

layouttype = 'orig' #globalmod' # 'spring' 'global' 'orig'

met = 'euclidean' # 'euclidean' # 'cosine'

radius = 1.0
NN = 10

sampled = 'volume' # 'surface' #'volume'

if sampled == 'volume':
    positions = sample_pts_spherevolume(n_samples,n_dimensions,radius)
    G = get_umap_network(positions,NN,met)
    d_pos_highdim = dict(zip(list(G.nodes()), positions))
    d_pos_highdim_norm = normalize_pos(d_pos_highdim)
elif sampled == 'surface':
    positions = sample_pts_spheresurface(n_samples,n_dimensions)
    G = get_umap_network(positions,NN,met)
    d_pos_highdim = dict(zip(list(G.nodes()), positions))
    d_pos_highdim_norm = normalize_pos(d_pos_highdim)



##############################################################

# HEATMAP

##############################################################

fig = pgo.Figure(data=pgo.Heatmap(z=positions, colorscale='Cividis'))
fig.update_layout(
    title='N-dim Sphere | Samples: ' + str(n_samples) + ' | Data dimensions: ' + str(n_dimensions) + ' | Sampled method: ' + str(sampled),
    xaxis_title='Features',
    yaxis_title='Samples',
    xaxis_nticks=n_dimensions,
    yaxis_nticks=n_samples,
    xaxis_tickangle=-45,
    width=1200,
    height=1200,
    xaxis=dict(
        tickvals=list(range(0, n_dimensions, 50)),  # Show every 5th tick label on the x-axis
        tickfont=dict(size=10)  # Adjust the font size for x-axis ticks
    ),
    yaxis=dict(
        tickvals=list(range(0, n_samples, 50)),  # Show every 5th tick label on the y-axis
        tickfont=dict(size=10)  # Adjust the font size for y-axis ticks
    )
)

file = 'NdimSphere-Heatmap_samples-' + str(n_samples) + '_features-' + str(n_dimensions) + '_method-' + str(sampled)
fig.write_html('plots/'+file+'.html')
fig.write_image('plots/'+file+'.png')
plotly.offline.plot(fig, filename = 'plots/'+file+'.html', auto_open=False)




##############################################################

# LAYOUT         

##############################################################

if layouttype == 'orig': # visualize original points 
    posG3D = d_pos_highdim_norm


elif layouttype == 'spring':
    posG3D_spring = nx.spring_layout(G,dim=3,iterations = 200)
    posG3D = dict(zip(G.nodes(),[list(i) for i in posG3D_spring.values()]))


elif layouttype == 'globalmod': # AIJ/SPL combined -> RWR 
    A = nx.adjacency_matrix(G)
    FM_adj = A.toarray()
    FM_spl = calculate_spl_matrix(G)

    # Normalize the feature matrices between 0 and 1 using Min-Max scaling
    min_max_scaler = MinMaxScaler()
    FM_spl_norm = min_max_scaler.fit_transform(FM_spl)
    matrix_combined =  combine_matrices(FM_adj, 0.5, FM_spl_norm, 0.5)

    # RWR - FEATURE MATRIX
    r = .8 
    alpha = 1.0  
    FM_rwrmod = random_walk_with_restart(matrix_combined, r, alpha, len(G.nodes()))
    columns = [i for i in range(FM_rwrmod.shape[1])]
    df_rwrmod = pd.DataFrame(data=FM_rwrmod, columns=columns)

    n_n = 10 # this should be constant
    spr = 1.0  
    md = 0.0  
    posG3D = layout_functional_umap(G,
                                df_rwrmod,
                                dim=3,
                                n_neighbors=n_n, 
                                spread=spr, 
                                min_dist=md, 
                                metric=met)

elif layouttype == 'global':
    A = nx.adjacency_matrix(G)
    adj = A.toarray()
    DM_adj = pd.DataFrame(adj, columns = list(G.nodes()), index=list(G.nodes()))

    r = .8 # restart probability; originally 0.8
    alpha = 1.0 # alpha = 0.5 # --> better results for torus 

    DM_m = random_walk_with_restart(A, r, alpha, len(G.nodes()))

    DM_m_df = pd.DataFrame(DM_m)
    #print(DM_m_df.sum(axis=0))
    DM_m_transposed = DM_m_df.T

    DM_m_new = pd.DataFrame(distance.squareform(distance.pdist(DM_m_transposed, met)))
    DM_m_new.index = list(G.nodes())
    DM_m_new.columns = list(G.nodes())
    DM_m = DM_m_new

    n_n = 10 # this should be constant
    spr = 1.0  
    md = 0.0  
    posG3D = layout_functional_umap(G,
                                DM_m,
                                dim=3,
                                n_neighbors=n_n, 
                                spread=spr, 
                                min_dist=md, 
                                metric=met)



##############################################################

# VISUALIZATION 

##############################################################



opacity_nodes = 0.9
node_edge_col = '#696969' 
node_linewidth = 0.25

edge_opacity = 0.1
edge_width = 2
edge_color = '#696969'  ##7D7D7D' # '#d3d3d3'

scale_factor = 0.1
scale_factor_log = 2.25
#d_size = dict(zip(list(G.nodes()),[5]*len(G.nodes())))
d_size = dict(draw_node_degree(G, scale_factor))
d_size_log = dict(draw_node_degree_log(G, scale_factor_log))



# COLOR BY distance to center
colvalue = 'dist2center'

if sampled == 'volume':
    xdim_centroid = get_centroid_datacloud(d_pos_highdim_norm)
elif sampled == 'surface':
    xdim_centroid = get_centroid_datacloud(d_pos_highdim_norm) #[0.5,0.5,0.5]

d_xdim_centroid = {}
d_xdim_centroid[0] = [i for i in xdim_centroid]
l_distances_p2c_norm = calculate_distance_pts2center(n_dimensions,xdim_centroid,d_pos_highdim_norm)

colors_cmap = matplotlib.cm.coolwarm_r #Blues_r #_r
#plot_colorlegend(colors_cmap)

# sort color map values according to distances and make dictionary for trace function --> to be sorted by Graph nodes 
colors_cmap_sortedbyG = {}
for i in l_distances_p2c_norm:
    colors_cmap_sortedbyG[i] = colors_cmap(i)

# rgba to hex
colors_cmap_sortedbyG_hex = {}
for dist,col in colors_cmap_sortedbyG.items():
    colors_cmap_sortedbyG_hex[dist] = matplotlib.colors.rgb2hex(col)
    
# actual color dictionary sorted according to Graph nodes 
d_colors = dict(zip(list(G.nodes()),colors_cmap_sortedbyG_hex.values()))

d_node_annotations = dict(nx.degree(G))
l_node_annotations = list(d_node_annotations.values())



node_centroid = get_trace_nodes_3D_legend(d_xdim_centroid,
                                   dict(zip(list(d_xdim_centroid.keys()),d_xdim_centroid.keys())),
                                   dict(zip(list(d_xdim_centroid.keys()),['#000000'])),
                                   dict(zip(list(d_xdim_centroid.keys()),[10])))
nodes = get_trace_nodes_3D_legend(posG3D, 
                                  dict(zip(list(G.nodes()),posG3D.keys())), 
                                  d_colors, 
                                  d_size_log,
                                 )
edges = get_trace_edges_3D(G, 
                           posG3D,
                           color=edge_color,
                           opac=edge_opacity,
                           linewidth=edge_width
                          )


alphahull_val = 10 # the smaller the more likely a structure covers the whole network 

mesh = pgo.Mesh3d(alphahull = alphahull_val,
                        x = [v[0] for k,v in posG3D.items()],
                        y = [v[1] for k,v in posG3D.items()],
                        z = [v[2] for k,v in posG3D.items()],
                        color = 'steelblue',
                        opacity = 0.05,
        name='mesh'
)



data = [
        #mesh,
        edges,
        *nodes,
        *node_centroid 
       ]

scheme = 'light'
file = 'Sphere_'+sampled+'_layout-'+layouttype+'_Ndim'+str(n_dimensions)+'metr-'+met+'_samples'+str(n_samples)+'_graphNN'+str(NN)+'_col-'+colvalue+'_scheme-'+scheme

plot_3D(data,path_plots,file,scheme) # annotat = d_node_annotations)
#plot_3D_animated(data,path,file,scheme)

### include mesh 

In [ ]:
# including mesh
alphahull_val = 8 # the smaller the more likely a structure covers the whole network 
mesh = pgo.Mesh3d(alphahull = alphahull_val,
                        x = [v[0] for k,v in posG3D.items()],
                        y = [v[1] for k,v in posG3D.items()],
                        z = [v[2] for k,v in posG3D.items()],
                        color = 'steelblue',
                        opacity = 0.035, name='mesh'
                        )

data = [
        #mesh,
        edges,
        *nodes
       ]

### Spectral layout

In [ ]:
%%time 

layouttype = 'spectral'

dim_spectral = 3
posGND = nx.spectral_layout(G,dim = dim_spectral)

df_posGND = pd.DataFrame(posGND).T
for col in df_posGND.columns:
    df_posGND[col] = min_max_scaling(df_posGND[col])
df_posGND_t = df_posGND.T
posG3D_spectral = df_posGND_t.to_dict(orient='list')

posG3D = posG3D_spectral

### Node2Vec layout

In [ ]:
layouttype = 'nodevec' 

G_sg = StellarGraph.from_networkx(G)
rw = BiasedRandomWalk(G_sg)

In [ ]:
walks = rw.run(
    nodes=list(G_sg.nodes()),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)

In [ ]:
str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, vector_size=128, window=5, min_count=0, sg=1, workers=2)

# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index_to_key  # list of node IDs
node_embeddings = (
    model.wv.vectors
)  

In [ ]:
n_n = 12 # this should be constant
spr = 1.0  
md = 0.0  

umap_3D = embed_umap_3D(node_embeddings, n_n, spr, md, met)
x_umap = [i[0] for i in umap_3D]
y_umap = [i[1] for i in umap_3D]
z_umap = [i[2] for i in umap_3D]

xx_norm3D = preprocessing.minmax_scale(x_umap, feature_range=(0, 1), axis=0, copy=True)
yy_norm3D = preprocessing.minmax_scale(y_umap, feature_range=(0, 1), axis=0, copy=True)
zz_norm3D = preprocessing.minmax_scale(z_umap, feature_range=(0, 1), axis=0, copy=True)

posG_nodevec = dict(zip(node_ids , zip(xx_norm3D,yy_norm3D,zz_norm3D)))

In [ ]:
posG3D = {key:posG_nodevec[key] for key in G_sg.nodes()}

In [ ]:
G_sg.nodes()

### node colors by degree

In [ ]:
d_degree = dict(zip(list(G.nodes()),l_size))
d_to_be_colored = d_degree

col_pal = 'Blues' 
#d_colors = dict(zip(list(G.nodes()),cm.viridis(list(magnitudes.flatten())))) 
d_colors = color_nodes_from_dict(G, d_to_be_colored, col_pal)
l_colors = list(d_colors.values())

### 2D plot

In [ ]:
plt.figure(figsize=(25,25))
plt.title(fname, size=16)

nx.draw_networkx_nodes(G, 
                       posG,
                       edgecolors = node_edge_col, 
                       linewidths = node_linewidth, 
                       node_color = l_colors, 
                       node_size = l_size, 
                       alpha = opacity_nodes)

nx.draw_networkx_edges(G, posG, width = edge_width, edge_color = edge_color, alpha = edge_opacity)
plt.box(False)

plt.savefig(path+fname+'.png',dpi=300)
plt.show()

In [ ]:
posG2D = nx.spectral_layout(G, dim=2)#,iterations=50)
posG3D = nx.spectral_layout(G, dim=3)#,iterations=50)

In [ ]:
plt.figure(figsize=(25,25))
plt.title(fname, size=16)

nx.draw_networkx_nodes(G, 
                       posG2D,
                       edgecolors = node_edge_col, 
                       linewidths = node_linewidth, 
                       #node_color = dict(cm.viridis(positions[0]>1), 
                       node_size = 50, 
                       alpha = opacity_nodes)

nx.draw_networkx_edges(G, posG2D, width = edge_width, edge_color = '#D3D3D3', alpha = 1.0)
plt.box(False)

plt.savefig(path+fname+'.png',dpi=300)
plt.show()

# other layouts

### Importance layout 

In [ ]:
%%time 

layouttype = 'importance'

n_n = NN
spr = 1.0  
md = 0.0  
met = 'euclidean' 

posG3D = layout_importance_umap(G,
                            dim=3,
                            n_neighbors=n_n, 
                            spread=spr, 
                            min_dist=md, 
                            metric=met)
df_posG3D = pd.DataFrame(posG3D).T

x3D=df_posG3D[0]
y3D=df_posG3D[1]
z3D=df_posG3D[2]

### Local layout

In [ ]:
%%time 

layouttype = 'local'

n_n = NN
spr = 1.0  
md = 0.0  
met = 'euclidean' 

posG3D = layout_local_umap(G,
                            dim=3,
                            n_neighbors=n_n, 
                            spread=spr, 
                            min_dist=md, 
                            metric=met)
df_posG3D = pd.DataFrame(posG3D).T

x3D=df_posG3D[0]
y3D=df_posG3D[1]
z3D=df_posG3D[2]

# OLD STUFF

In [ ]:
# ----------
# old 
# ----------
#x = [i[0] for i in list(posG3D.values())]
#y = [i[1] for i in list(posG3D.values())]
#z = [i[2] for i in list(posG3D.values())]
#center = sum(x)/len(x), sum(y)/len(y),sum(z)/len(z)
#print(center)

#distances = {}
#for k,i in d_pos_highdim.items():
#    x = i[0]
#    y = i[1]
#    z = i[2]
#    centerx = center[0]
#    centery = center[1]
#    centerz = center[2]
#    dist = math.sqrt(((x - centerx) ** 2) + ((y - centery) ** 2) + ((z - centerz) ** 2))
#    distances[k] = dist

#scaler = MinMaxScaler(feature_range=(0, 1))
#dist_scaled = scaler.fit_transform([[x] for x in distances.values()])

#cols = (matplotlib.cm.rainbow_r(dist_scaled))
#col = []
#for i in cols:
#    col.extend(i)
#cols_hex = [rgb2hex(int(i[0]*255),int(i[1]*255),int(i[2]*255)) for i in col]
#d_colors = dict(zip(list(G.nodes()),cols_hex))